# python asyncio 案例

## case 1:  协程函数和任务事件环
### 1.1 通过async关键字定义一个协程（coroutine）,当然协程不能直接运行，需要将协程加入到事件循环loop中
### 1.2 asyncio.get_event_loop：创建一个事件循环，然后使用run_until_complete将协程注册到事件循环，并启动事件循环
### 1.3 协程对象不能直接运行，由asyncio的loop.run_until_complete启动.

In [1]:
import asyncio
import time

# 我们通过async关键字定义一个协程,当然协程不能直接运行，需要将协程加入到事件循环loop中
async def do_some_work(x):
    print("waiting:", x)

start = time.time()

coroutine = do_some_work(2)
loop = asyncio.get_event_loop()        # asyncio.get_event_loop：创建一个事件循环
# 通过loop.create_task(coroutine)创建task,同样的可以通过 asyncio.ensure_future(coroutine)创建task
task = loop.create_task(coroutine)     # 创建任务, 不立即执行
loop.run_until_complete(task)         # 使用run_until_complete将协程注册到事件循环，并启动事件循环
print("Time:",time.time() - start)

Time: 0.0009999275207519531
waiting: 2


## case 2: 增加协程函数的返回值
### case 1 说明async def的协程函数可以接受输入参数
### case 2 将进一步设计async def的协程函数返回值

In [2]:
import asyncio
import time

# 我们通过async关键字定义一个协程,当然协程不能直接运行，需要将协程加入到事件循环loop中
async def do_some_work(x):
    print("waiting:", x)
    return "Done after {}s".format(x)

def callback(future):
    print("callback:",future.result())

start = time.time()

coroutine = do_some_work(2)
loop = asyncio.get_event_loop()        # asyncio.get_event_loop：创建一个事件循环
# 通过loop.create_task(coroutine)创建task,同样的可以通过 asyncio.ensure_future(coroutine)创建task
task = loop.create_task(coroutine)     # 创建任务, 不立即执行
# task = asyncio.ensure_future(coroutine)
task.add_done_callback(callback)
# 绑定回调，在task执行完成的时候可以获取执行的结果
loop.run_until_complete(task)         # 使用run_until_complete将协程注册到事件循环，并启动事件循环
print("Time:",time.time() - start)

Time: 0.0009999275207519531
waiting: 2
callback: Done after 2s


## case 3: 阻塞和await
### async def的协程函数的重要价值是可以针对一些不确定发生的过程通过关键词await主动让出cpu时间

### 作为对比，先给出一个3过程顺序执行例

In [ ]:
# 普通串行花费7秒
import time
def do_some_work(t):
    time.sleep(t)
    print('用了%s秒' % t)

start = time.time()
coroutine1 = do_some_work(1)
coroutine2 = do_some_work(2)
coroutine3 = do_some_work(4)
print(time.time()-start)

### 预期输出为
用了1秒
用了2秒
用了4秒
7.002151012420654
注意：3个顺序执行，最后总的用时为7秒

### 以下为并发执行例

In [ ]:
# 使用协程并发执行将只花费4秒
import asyncio
import time

async def do_some_work(x):
    print("Waiting:",x)
    await asyncio.sleep(x)                   # 将耗时的sleep 标记为 await，这样会主动让出cpu时间
    return "Done after {}s".format(x)

start = time.time()

coroutine1 = do_some_work(1)
coroutine2 = do_some_work(2)
coroutine3 = do_some_work(4)

tasks = [
    asyncio.ensure_future(coroutine1),       # task对象是Future类的子类，保存了协程运行后的状态，用于未来获取协程的结果
    asyncio.ensure_future(coroutine2),       # 与case2的 task = loop.create_task(coroutine) 
    asyncio.ensure_future(coroutine3)        # 兼容
]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))  # 

for task in tasks:
    print("Task ret:",task.result())

print("Time:",time.time() - start)

### 预期输出为
#### Waiting: 1
#### Waiting: 2
#### Waiting: 4
#### Task ret: Done after 1s
#### Task ret: Done after 2s
#### Task ret: Done after 4s
#### Time: 4.0038135051727295
#### 注意：3个并行执行，最后总的用时为4秒

## case 4: 协程嵌套

### 4.1 形式1

In [ ]:
import asyncio
import time

async def do_some_work(x):
    print("waiting:",x)
    await asyncio.sleep(x)
    return "Done after {}s".format(x)

async def main():
    coroutine1 = do_some_work(1)
    coroutine2 = do_some_work(2)
    coroutine3 = do_some_work(4)
    tasks = [
        asyncio.ensure_future(coroutine1),
        asyncio.ensure_future(coroutine2),
        asyncio.ensure_future(coroutine3)
    ]
    return await asyncio.wait(tasks)

start = time.time()

loop = asyncio.get_event_loop()
done,pending = loop.run_until_complete(main())
for task in done:
    print("Task ret:",task.result())

print("Time:", time.time() - start)


### 4.1 形式2
使用列表推导式简写

In [ ]:
import time
import asyncio

async def job(t):            # 使用 async 关键字将一个函数定义为协程
    await asyncio.sleep(t)   # 等待 t 秒, 期间切换执行其他任务
    print('用了%s秒' % t)

async def main(loop):           # 使用 async 关键字将一个函数定义为协程
    tasks = [loop.create_task(job(t)) for t in range(1,3)]  # 创建任务, 不立即执行
    await asyncio.wait(tasks)   # 执行并等待所有任务完成

start = time.time()
loop = asyncio.get_event_loop()      # 创建一个事件loop
loop.run_until_complete(main(loop))  # 将事件加入到事件循环loop
loop.close()                         # 关闭 loop

print(time.time()-start)
